In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.stats

In [26]:
x = np.array([4, 6, 9, 9, 11, 12, 19, 22, 27, 30, 33, 36, 39])
y = np.array([44, 66, 81, 99, 111, 122, 139, 103, 22, 17])
t = x[:-3] #Тренировочные данные

In [33]:
from gekko import GEKKO    
import sklearn.metrics


n = 4 #кол-во переменных
m = GEKKO(remote=False) # create model

m.options.SOLVER=1  #solver for nonlinear tasks

#Изменяемые значения
H = m.Var(value = 100) 
A = m.Var(value = 50)
W = m.Var(value = 6.28/30 )
f = m.Var(value = -2)
#Целевая функция = sum(e^2) -> min
obj = 0
for i in range(len(t)):
    obj += (y[i]-(H+A*m.sin(t[i]*W+f)))**2
m.Minimize(obj) #целевая фунция
m.solve() 
res = np.array([H.value[0], A.value[0], W.value[0], f.value[0]])
print(res) # print solution
y_pred = res[0]+res[1]*np.sin(x*res[2]+res[3])
e = y-y_pred[:-3]
R_2 = sklearn.metrics.r2_score(y, y_pred[:-3])
results = pd.DataFrame({'t': t, 'Y': y, 'Y^': y_pred[:-3], 'e': e, 'e^2': e**2}) 
results

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            8
   Intermediates:            0
   Connections  :            0
   Equations    :            1
   Residuals    :            1
 
 Number of state variables:              8
 Number of total equations: -            0
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              8
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  7.71805E+13  0.00000E+00
    1  2.55450E+03  0.00000E+00
    2  1.30494E+03  0.00000E+00
    3  1.08009E+03  0.00000E+00
    4  

,t,Y,Y^,e,e^2
0,4,44,34.615387,9.384613,88.070960
1,6,66,56.271520,9.728480,94.643314
2,9,81,94.178144,-13.178144,173.663490
3,9,99,94.178144,4.821856,23.250291
4,11,111,117.246393,-6.246393,39.017426
5,12,122,126.697016,-4.697016,22.061962
6,19,139,129.017726,9.982274,99.645794
7,22,103,97.648762,5.351238,28.635752
8,27,22,37.299437,-15.299437,234.072780
9,30,17,16.845558,0.154442,0.023852


In [37]:
#Прогноз для последних трех значений 
x[-3:], y_pred[-3:]

(array([33, 36, 39]), array([18.03366687, 40.44848851, 76.25546301]))